In [1]:
import numpy as np
import pandas as pd
import sys
from pathlib import Path
sys.path.append('..') 
import platform
import os
from pathlib import Path
import shutil
from pathlib import Path
import shutil
from utils.behavior.session_utils import load_session_df, parse_session_string
from utils.behavior.lick_analysis import clean_up_licks, parse_lick_trains
from scipy.io import loadmat
from itertools import chain
from matplotlib import pyplot as plt
from IPython.display import display
from scipy.signal import find_peaks
from harp.clock import align_timestamps_to_anchor_points
from utils.basics.data_org import *
from utils.photometry.preprocessing import * 
from utils.photometry.plot_utils import align_signal_to_events, color_gradient, plot_FP_with_licks
import numpy as np
from scipy.signal import butter, filtfilt, medfilt
from scipy.optimize import curve_fit
import json
from sklearn.linear_model import LinearRegression
from matplotlib.gridspec import GridSpec
import pickle
from aind_fip_dff.utils.preprocess import batch_processing, tc_triexpfit
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
import time
from contextlib import redirect_stdout
# %matplotlib inline
# %matplotlib widget

### Signle session

In [8]:
session = 'm699462d20240104'
label = 'CombineWithKH'
session_dir = parse_session_string(session)
session_df, licks_L, licks_R = load_session_df(session)
licks_L, licks_R, fig = clean_up_licks(licks_L, licks_R, plot=False)
parsed_licks_L, _ = parse_lick_trains(licks_L)
parsed_licks_R, _ = parse_lick_trains(licks_R)
signal_region = load_session_FP(session, label, plot=False);
signal_region_raw = load_session_FP_raw(session, label, plot=False);
# signal_pre = preprocess_signal(session, label, fs = 20, lowcut = 0.1, fc = 9, xtol=1e-6, plot=True)

Region mPFC recorded at fiber 0
Region CrusI recorded at fiber 1
Region LC recorded at fiber 2
Region mPFC recorded at fiber 0
Region CrusI recorded at fiber 1
Region LC recorded at fiber 2
Channel G:Region mPFC recorded at fiber 0
Channel G:Region CrusI recorded at fiber 1
Channel G:Region LC recorded at fiber 2
Channel Iso:Region mPFC recorded at fiber 0
Channel Iso:Region CrusI recorded at fiber 1
Channel Iso:Region LC recorded at fiber 2


#### Add CO version to current version

In [ ]:
signal_region_prep_CO, params, figs = preprocess_signal_CO(session, label, plot=True, xtol=1e-10, methods=['tri-exp'])
append_FP_data(session, label, signal_region_prep_CO, params)

Loaded m699461d20231222_combined.pkl and m699461d20231222_combined_params.pkl
Finished writing m699461d20231222_combined.pkl
Finished writing m699461d20231222_combined_params.pkl


#### Load all preprocessed data

In [ ]:
signal_region_prep_updated, params = get_FP_data(session, label)
list(signal_region_prep_updated['G'].keys())

### Batch processing

In [2]:
session_list = get_session_list('photometry', 'all', 'crusIgood')
label = 'CombineWithKH'

In [8]:
session_list = get_session_list('photometry_Gi', 'all', 'Gi_G8')
label = 'Gi'

#### If there's no preprocessing at all

In [5]:
for session in session_list:
    get_FP_data(session, label)

Loaded m699461d20231217_combined.pkl and m699461d20231217_combined_params.pkl
Loaded m699461d20231218_combined.pkl and m699461d20231218_combined_params.pkl
Loaded m699461d20231219_combined.pkl and m699461d20231219_combined_params.pkl
Loaded m699461d20231220_combined.pkl and m699461d20231220_combined_params.pkl
Loaded m699461d20231222_combined.pkl and m699461d20231222_combined_params.pkl
Loaded m699461d20240102_combined.pkl and m699461d20240102_combined_params.pkl
Loaded m699461d20240103_combined.pkl and m699461d20240103_combined_params.pkl
Loaded m699461d20240104_combined.pkl and m699461d20240104_combined_params.pkl
Loaded m699461d20240105_combined.pkl and m699461d20240105_combined_params.pkl
Loaded m699461d20240106_combined.pkl and m699461d20240106_combined_params.pkl
Loaded m699461d20240107_combined.pkl and m699461d20240107_combined_params.pkl
Loaded m699461d20240116_combined.pkl and m699461d20240116_combined_params.pkl
Loaded m699461d20240117_combined.pkl and m699461d20240117_combin

#### If already preprocessed: Add CO version preprocessing to existing versions

In [9]:
def process_session(session):
    start_time = time.time()  # Record start time
    session_dir = parse_session_string(session)
    
    # Define the output file path
    output_file = os.path.join(session_dir['sortedFolder'], f'{session}_output.txt')
    
    # Ensure the output directory exists
    os.makedirs(session_dir['sortedFolder'], exist_ok=True)
    
    # Open the output file in append mode
    with open(output_file, 'a') as f:
        with redirect_stdout(f):  # Redirect all print statements to the file
            print(f"Processing {session}")
            try:
                signal_region_prep_CO, params, figs = preprocess_signal_CO(session, label, plot=True, xtol=1e-8, methods=['tri-exp'])
                append_FP_data(session, label, signal_region_prep_CO, params)
                plt.close('all')
                print(f"Processing complete for {session}.")

            except Exception as e:
                print(f"An error occurred while processing {session}: {e}")
            finally:
                # Log the processing time
                end_time = time.time()
                elapsed_time = end_time - start_time
                print(f"Processing time for {session}: {elapsed_time:.2f} seconds")

# Run in parallel using joblib
Parallel(n_jobs=8)(delayed(process_session)(session) for session in session_list)
# process_session(session_list[0])


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

#### File migration

In [39]:
# file migration from pickle to json
for session in session_list:
    print(session)
    session_dir = parse_session_string(session)
    old_file = os.path.join(session_dir['sortedFolder'], f'{session}_combined_params.pkl')
    with open(old_file, 'rb') as f:
        old_params = pickle.load(f)
    new_file = os.path.join(session_dir['sortedFolder'], f'{session}_combined_params_updated.json')
    with open(new_file, 'w') as f:
        json.dump(old_params, f)
    print(f'Finished changing {session}')

m699462d20240113
Loaded m699462d20240113_FP_CombinewithKH.pkl
Loaded m699462d20240113_FP_CombinewithKH.pkl
Loaded m699462d20240113_FP_CombinewithKH.pkl
Loaded m699462d20240113_FP_CombinewithKH.pkl
m699462d20240114
Loaded m699462d20240114_FP_CombinewithKH.pkl
Loaded m699462d20240114_FP_CombinewithKH.pkl
Loaded m699462d20240114_FP_CombinewithKH.pkl
Loaded m699462d20240114_FP_CombinewithKH.pkl
m699462d20240116
Loaded m699462d20240116_FP_CombinewithKH.pkl
Loaded m699462d20240116_FP_CombinewithKH.pkl
Loaded m699462d20240116_FP_CombinewithKH.pkl
Loaded m699462d20240116_FP_CombinewithKH.pkl
m699462d20240117
Loaded m699462d20240117_FP_CombinewithKH.pkl
Loaded m699462d20240117_FP_CombinewithKH.pkl


c:\Users\zhixi\Documents\GitHub\aind_su_etal_2022\notebooks\..\utils\photometry\plot_utils.py:60: RuntimeWarning: Mean of empty slice
  aligned_matrix[i, j] = np.nanmean(aligned_signal[window_mask])


Loaded m699462d20240117_FP_CombinewithKH.pkl


c:\Users\zhixi\Documents\GitHub\aind_su_etal_2022\notebooks\..\utils\photometry\plot_utils.py:60: RuntimeWarning: Mean of empty slice
  aligned_matrix[i, j] = np.nanmean(aligned_signal[window_mask])


Loaded m699462d20240117_FP_CombinewithKH.pkl


...

Loaded m701707d20240105_FP_CombinewithKH.pkl


c:\Users\zhixi\Documents\GitHub\aind_su_etal_2022\notebooks\..\utils\photometry\plot_utils.py:60: RuntimeWarning: Mean of empty slice
  aligned_matrix[i, j] = np.nanmean(aligned_signal[window_mask])


Loaded m701707d20240105_FP_CombinewithKH.pkl


c:\Users\zhixi\Documents\GitHub\aind_su_etal_2022\notebooks\..\utils\photometry\plot_utils.py:60: RuntimeWarning: Mean of empty slice
  aligned_matrix[i, j] = np.nanmean(aligned_signal[window_mask])


m701707d20240106
Loaded m701707d20240106_FP_CombinewithKH.pkl
Loaded m701707d20240106_FP_CombinewithKH.pkl
Loaded m701707d20240106_FP_CombinewithKH.pkl
Loaded m701707d20240106_FP_CombinewithKH.pkl
m701707d20240112
Loaded m701707d20240112_FP_CombinewithKH.pkl
Loaded m701707d20240112_FP_CombinewithKH.pkl
Loaded m701707d20240112_FP_CombinewithKH.pkl
Loaded m701707d20240112_FP_CombinewithKH.pkl
m701707d20240113
Loaded m701707d20240113_FP_CombinewithKH.pkl
Loaded m701707d20240113_FP_CombinewithKH.pkl
Loaded m701707d20240113_FP_CombinewithKH.pkl
Loaded m701707d20240113_FP_CombinewithKH.pkl
m701707d20240114
Loaded m701707d20240114_FP_CombinewithKH.pkl
Loaded m701707d20240114_FP_CombinewithKH.pkl
Loaded m701707d20240114_FP_CombinewithKH.pkl
Loaded m701707d20240114_FP_CombinewithKH.pkl
m701707d20240117
Loaded m701707d20240117_FP_CombinewithKH.pkl
Loaded m701707d20240117_FP_CombinewithKH.pkl
Loaded m701707d20240117_FP_CombinewithKH.pkl
Loaded m701707d20240117_FP_CombinewithKH.pkl


In [19]:
# File migration between locations
for session in session_list:
    session_dir = parse_session_string(session)
    # move signal
    old_file = os.path.join(session_dir['photometryPath'], f'{session}_combined.pkl')
    new_folder = session_dir['sortedFolder']
    shutil.move(old_file, new_folder)
    print(f"Finished moving {session}_combined.pkl")
    # move params
    old_file = os.path.join(session_dir['photometryPath'], f'{session}_combined_params.pkl')
    new_folder = session_dir['sortedFolder']
    shutil.move(old_file, new_folder)
    print(f"Finished moving {session}_combined_params.pkl")

Finished moving m749624d20250102_combined.pkl
Finished moving m749624d20250102_combined_params.pkl
Finished moving m749624d20250103_combined.pkl
Finished moving m749624d20250103_combined_params.pkl
Finished moving m749624d20250104_combined.pkl
Finished moving m749624d20250104_combined_params.pkl
Finished moving m749624d20250107_combined.pkl
Finished moving m749624d20250107_combined_params.pkl
Finished moving m749624d20250109_combined.pkl
Finished moving m749624d20250109_combined_params.pkl
Finished moving m749472d20241228_combined.pkl
Finished moving m749472d20241228_combined_params.pkl
Finished moving m749472d20241230_combined.pkl
Finished moving m749472d20241230_combined_params.pkl
Finished moving m749472d20241231_combined.pkl
Finished moving m749472d20241231_combined_params.pkl
Finished moving m749472d20250101_combined.pkl
Finished moving m749472d20250101_combined_params.pkl
Finished moving m749472d20250103_combined.pkl
Finished moving m749472d20250103_combined_params.pkl
Finished m

### Plot lick artifact

In [ ]:
def process(session):
    # write tmp_dict to json
    signal_region_prep_updated, params = get_FP_data(session, label)
    regions = list(signal_region_prep_updated['G'].keys())
    for region in regions:
        plot_FP_with_licks(session, label, region)
    plt.close('all')
Parallel(n_jobs=8)(delayed(process)(session) for session in session_list[1:])
# process(session_list[0])

In [31]:
# def plot_FP_with_licks_test(session, label, region):
#     session_df, licks_L, licks_R = load_session_df(session)
#     session_dir = parse_session_string(session)
#     signal_region_prep, params = get_FP_data(session, label)
#     licks_L, licks_R, fig = clean_up_licks(licks_L, licks_R, plot=False)
#     parsed_licks_L, _ = parse_lick_trains(licks_L)
#     parsed_licks_R, _ = parse_lick_trains(licks_R)
#     trial_starts = session_df['CSon']
#     licks_in_trial_L = [train_start for train_start in list(parsed_licks_L['train_starts']) if any([trial_start<train_start and trial_start>train_start-2000  for trial_start in trial_starts])]
#     licks_in_trial_R = [train_start for train_start in list(parsed_licks_R['train_starts']) if any([trial_start<train_start and trial_start>train_start-2000  for trial_start in trial_starts])]
#     licks_out_trial_L = [train_start for train_start in list(parsed_licks_L['train_starts']) if not any([trial_start<train_start and trial_start>train_start-2000  for trial_start in trial_starts])]
#     licks_out_trial_R = [train_start for train_start in list(parsed_licks_R['train_starts']) if not any([trial_start<train_start and trial_start>train_start-2000  for trial_start in trial_starts])]
#     fig = plt.figure(figsize=(15, 40))
#     colorL = 'b'
#     colorR = 'r'
#     all_channels = [key for key, value in signal_region_prep.items() if 'time' not in key]
#     gs = GridSpec(len(all_channels), 5, figure=fig)
#     for channel_id, channel in enumerate(all_channels):
#         signal = signal_region_prep[channel][region]
#         ax = fig.add_subplot(gs[channel_id, 0])
#         align_signal_to_events(signal, signal_region_prep['time_in_beh'], parsed_licks_L['train_starts'], ax = ax, legend = 'L', color = colorL)
#         align_signal_to_events(signal, signal_region_prep['time_in_beh'], parsed_licks_R['train_starts'], ax = ax, legend = 'R', color = colorR)
#         ax.legend()
#         ax.set_title(f'All licks')
#         ax.set_ylabel(channel)
#         # in vs out trial L
#         ax = fig.add_subplot(gs[channel_id, 1])
#         align_signal_to_events(signal, signal_region_prep['time_in_beh'], licks_in_trial_L, ax = ax, color = colorL, legend = 'in')
#         align_signal_to_events(signal, signal_region_prep['time_in_beh'], licks_out_trial_L, ax = ax, color = colorR, legend = 'out')
#         ax.legend()
#         ax.set_title(f'In vs out trial L')
#         # in vs out trial R
#         ax = fig.add_subplot(gs[channel_id, 2])
#         align_signal_to_events(signal, signal_region_prep['time_in_beh'], licks_in_trial_R, ax = ax, color = colorL, legend = 'in')     
#         align_signal_to_events(signal, signal_region_prep['time_in_beh'], licks_out_trial_R, ax = ax, color = colorR, legend = 'out')
#         ax.legend()
#         ax.set_title(f'In vs out trial R')
#         # in left, in vs out trial with gradient of lick lick peak
#         ax = fig.add_subplot(gs[channel_id, 3])
#         num_bins = 3
#         peaks = parsed_licks_L['train_amps']
#         colors_in = color_gradient([1, 0, 0], num_bins+1)
#         colors_out = color_gradient([0, 0, 1], num_bins+1)
#         edges = np.quantile(peaks, np.linspace(0, 1, num_bins+1))
#         for ind in range(num_bins):
#             mask = (peaks>edges[ind]) & (peaks<=edges[ind+1])
#             if np.sum(mask)>2:
#                 align_signal_to_events(signal, signal_region_prep['time_in_beh'], np.array(parsed_licks_L['train_starts'])[mask], ax = ax, color = colors_in[ind+1], legend = f'In trial bin {ind}', plot_error=False)
#         ax.set_title(f'Left licks by lick peak')
#         # in right, in vs out trial with gradient of lick lick peak\
#         ax = fig.add_subplot(gs[channel_id, 4])
#         peaks = parsed_licks_R['train_amps']
#         colors_in = color_gradient([0, 0, 1], num_bins+1)
#         colors_out = color_gradient([0, 0, 1], num_bins+1)
#         edges = np.quantile(peaks, np.linspace(0, 1, num_bins+1))
#         edges[0] = edges[0]-0.01
#         for ind in range(num_bins):
#             mask = (peaks>edges[ind]) & (peaks<=edges[ind+1])
#             if np.sum(mask)>2:
#                 align_signal_to_events(signal, signal_region_prep['time_in_beh'], np.array(parsed_licks_R['train_starts'])[mask], ax = ax, color = colors_in[ind+1], legend = f'In trial bin {ind}', plot_error=False)  
#         ax.set_title(f'Right licks by lick peak')
#     plt.suptitle(f'{session}_{region}')
#     plt.tight_layout()

#     fig.savefig(os.path.join(session_dir['saveFigFolder'], f'{session}_{region}_FP_licks.pdf'))